In [2]:
# imports
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from tqdm import tqdm
from itertools import product
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import torch.nn.utils.rnn as RNN
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 42

data = pd.read_csv('data.csv', delimiter=';')
data.keys()

Index(['RECORDING_SESSION_LABEL', 'trial', 'IA_ID', 'item', 'list', 'IA_LABEL',
       'wordlength', 'condition', 'is_critical', 'is_spill1', 'is_spill2',
       'is_spill3', 'filler', 'LF', 'HF', 'function_word', 'other_filler',
       'composite', 'fixation_duration', 'duration_firstpass',
       'duration_firstfixation', 'fix_count', 'avg_pupil',
       'IA_REGRESSION_IN_COUNT', 'IA_REGRESSION_OUT_COUNT', 'saccade_length',
       'saccade_duration', 'go_past_time', 'sentenceCondition'],
      dtype='object')

In [3]:
filtered = data.copy()

filtered["sentenceCondition"] = filtered["sentenceCondition"].map(lambda x: x.replace("none", "2"))
filtered["sentenceCondition"] = filtered["sentenceCondition"].map(lambda x: x.replace("control", "0"))
filtered["sentenceCondition"] = filtered["sentenceCondition"].map(lambda x: x.replace("pseudo", "1"))
filtered["sentenceCondition"] = filtered["sentenceCondition"].map(lambda x: x.replace("filler", "3"))

filtered['attention'] = filtered['condition'].copy()

filtered['condition'] = filtered['condition'].map(lambda x: x.replace("control", "0"))
filtered['condition'] = filtered['condition'].map(lambda x: x.replace("pseudo", "1"))
filtered['condition'] = filtered['condition'].map(lambda x: x.replace("filler", "2"))
filtered['condition'] = filtered['condition'].map(lambda x: x.replace("none", "3"))

filtered['attention'] = filtered['attention'].map(lambda x: x.replace("control", "1"))
filtered['attention'] = filtered['attention'].map(lambda x: x.replace("pseudo", "1"))
filtered['attention'] = filtered['attention'].map(lambda x: x.replace("filler", "0"))
filtered['attention'] = filtered['attention'].map(lambda x: x.replace("none", "0"))


filtered["sentenceCondition"] = filtered["sentenceCondition"].astype(int)
filtered['condition'] = filtered['condition'].astype(int)
filtered['attention'] = filtered['attention'].astype(int)

control = filtered.loc[filtered['sentenceCondition'] == 0].copy()
pseudo = filtered.loc[filtered['sentenceCondition'] == 1].copy()
mapped = pd.concat([control, pseudo])


mapped.drop(["IA_ID", "item", "list", "IA_LABEL", "wordlength", "is_critical", 
              'is_spill1', 'is_spill2', 'is_spill3', 'filler', 'LF', 'HF', 'function_word', 'other_filler', "composite"], axis=1, inplace=True)
normalized = mapped[['fixation_duration',
       'duration_firstpass', 'duration_firstfixation', 'fix_count',
       'avg_pupil', 'IA_REGRESSION_IN_COUNT', 'IA_REGRESSION_OUT_COUNT',
       'saccade_length', 'saccade_duration', 'go_past_time']]
normalized = (normalized - normalized.mean()) / normalized.std()
mapped[['fixation_duration',
       'duration_firstpass', 'duration_firstfixation', 'fix_count',
       'avg_pupil', 'IA_REGRESSION_IN_COUNT', 'IA_REGRESSION_OUT_COUNT',
       'saccade_length', 'saccade_duration', 'go_past_time']] = normalized
sentences = mapped.groupby(['RECORDING_SESSION_LABEL', 'trial'])
print(len(sentences))  

343


In [4]:
label_array = list()
features_array = list()
attention_mask_array = list()
for item in sentences:
    label_array.append(item[1]["condition"])
    attention_mask_array.append(item[1]['attention'])
    features = item[1].drop(['RECORDING_SESSION_LABEL', 'trial', 'sentenceCondition', 'condition'], axis=1).to_numpy()
    features = (features - features.mean()) / features.std()
    #print(features.shape)
    features_array.append(features)

def pad_matrix_to_same_size(lists):
    maxlen = max([len(l) for l in lists])
    return [np.concatenate((np.zeros((maxlen - l.shape[0], l.shape[1])), l), axis=0) for l in lists]

def pad_series_to_same_size(lists):
    maxlen = max([len(l) for l in lists])
    return [np.concatenate((np.zeros((maxlen - len(l))), l), axis=0) for l in lists]

lengths = np.array([len(l) for l in features_array])
padded_features_array = np.array(pad_matrix_to_same_size(features_array))
padded_attention_mask_array = np.array(pad_series_to_same_size(attention_mask_array))
padded_label_array = np.array(pad_series_to_same_size(label_array))
print(padded_label_array.shape, padded_features_array.shape, padded_attention_mask_array.shape, lengths.shape)

(343, 18) (343, 18, 11) (343, 18) (343,)


In [5]:
class CustomDataset(Dataset):
    def __init__(self, features, labels, attention_mask):
        self.features = features
        self.labels = labels
        self.attention_mask = attention_mask

    def __getitem__(self, index):
        features = self.features[index]
        label = self.labels[index]
        attention_mask = self.attention_mask[index]
        return features, label, attention_mask

    def __len__(self):
        return len(self.features)

dataset = CustomDataset(features=padded_features_array, labels=padded_label_array, attention_mask=padded_attention_mask_array)

In [6]:
print(len(dataset))

343


In [7]:
def randomly_split_data(dataset, batch_size):
    

    #generator = torch.Generator().manual_seed(42)
    #train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1], generator=generator)
    train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])

    y = torch.tensor([label for _, label, _ in train_dataset], dtype=torch.long)

    # global class_weights
    # class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y.numpy())
    # class_weights = torch.tensor(class_weights, dtype=torch.float)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader, validation_dataloader, test_dataloader

In [8]:
def k_fold_split_data(dataset, batch_size, k=5):
    n = len(dataset)
    fold_size = n // k
    folds = []
    for i in range(k):
        start = i * fold_size
        end = (i + 1) * fold_size if i < k - 1 else n
        folds.append(torch.utils.data.Subset(dataset, range(start, end)))

    dataloaders = []
    for i in range(k):
        validation_dataset = folds[i]
        train_folds = [folds[j] for j in range(k) if j != i]
        train_dataset = torch.utils.data.ConcatDataset(train_folds)

        y = torch.tensor([label for _, label, _ in train_dataset], dtype=torch.long)

        # global class_weights
        # class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y.numpy())
        # class_weights = torch.tensor(class_weights, dtype=torch.float)

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
        dataloaders.append((train_dataloader, validation_dataloader))

    return dataloaders

In [9]:
def train_test(model, dataloader, optimizer, training="train"):
   
    loss_function = torch.nn.CrossEntropyLoss()#weight=class_weights.to(device))

    if training == "train":
        model.train()
    elif training == "validation":
        model.eval()
    elif training == "test":
        model.eval()
    else:
        raise ValueError("training argument must be either 'train', 'validation' or 'test'")
        
    cumulative_loss = 0
    prediction_list = []
    label_list = []
    for sample in dataloader:
        input, targets, attention_mask = sample[0].float().to(device), sample[1].type(torch.LongTensor).to(device) , sample[2].to(device)
        output = model(input, attention_mask).to(device)
        predictions = output[attention_mask == 1]
        targets = targets[attention_mask == 1]
        loss_value = loss_function(predictions, targets)
        cumulative_loss += loss_value.item()

        if training == "train":
            optimizer.zero_grad()
            loss_value.sum().backward()
            optimizer.step()
            
        predictions = predictions.to('cpu').detach().numpy().argmax(axis=1)
        
        target_labels = sample[1][attention_mask.to('cpu') == 1]
        prediction_list.extend(predictions)
        label_list.extend(target_labels)
    f1 = f1_score(label_list, prediction_list)
    accuracy = accuracy_score(label_list, prediction_list)
    confusion = confusion_matrix(label_list, prediction_list)

    return cumulative_loss, accuracy, f1, confusion

In [10]:
class TuneableModel(torch.nn.Module):
    def __init__(self, input_size, layer_size, dropout_rate, n_layers):
        super(TuneableModel, self).__init__()
        self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=layer_size, bidirectional=False, 
                                  num_layers=n_layers, batch_first=True, dropout=dropout_rate, proj_size=2)
        self.output_layer = torch.nn.Linear(layer_size*2, 2)
        self.batchnorm = torch.nn.BatchNorm1d(layer_size)
        self.activation = torch.nn.ReLU()
        self.linear = torch.nn.Linear(layer_size, layer_size)

    def forward(self, x, attention_mask):
        #print(x.shape)
        x = self.lstm(x)
        x = x[0]
        #x = self.output_layer(x)
        #print(x.shape)
        # x = self.batchnorm(x)
        # x = self.activation(x)
        # x = self.linear(x)
        # x = self.activation(x)
        return x

In [11]:
# Training sample
def evaluate(params):
    dropout, hidden_size, learning_rate, batch_size, n_hidden = params

    max_epochs = 1000
    max_patience = 10
    

    accuracies = []
    f1s = []
    #train_dataloader, validation_dataloader, test_dataloader = split_data(dataset, batch_size)
    dataloaders = k_fold_split_data(dataset, batch_size, k=10)
    for i, dataloader in tqdm(enumerate(dataloaders)):
        train_dataloader, validation_dataloader = dataloader[0], dataloader[1]
        test_dataloader = dataloader[1]
        PATH = f"model_{i}.pt"
        last_loss = 1000000
        torch.manual_seed(seed)
        input_size = train_dataloader.dataset[0][0].shape[1]
        model = TuneableModel(input_size, hidden_size, dropout, n_hidden)
        model.to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

        for epoch in range(max_epochs):
            # training
            train_loss, train_accuracy, train_f1, train_confusion = train_test(model, train_dataloader, optimizer, training="train")
            train_loss, train_accuracy, train_f1 = train_loss, round(train_accuracy, 4), round(train_f1, 2)
            # validation at end of epoch
            validation_loss, validation_accuracy, validation_f1, validation_confusion = train_test(model, validation_dataloader, optimizer, training="validation")
            validation_loss, validation_accuracy, validation_f1 = validation_loss, round(validation_accuracy, 4), round(validation_f1, 2)
            if validation_loss < last_loss:
                last_loss = validation_loss
                current_patience = 0
            else:
                if current_patience == 0:
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': last_loss,
                        }, PATH)
                current_patience += 1
            if current_patience == max_patience:
                break   

        # Testing once patience is reached
        torch.manual_seed(seed)
        model = TuneableModel(input_size, hidden_size, dropout, n_hidden)
        model.to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        checkpoint = torch.load(PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        test_loss, test_accuracy, test_f1, test_confusion = train_test(model, test_dataloader, optimizer, training="test")
        test_loss, test_accuracy, test_f1 = test_loss, test_accuracy, test_f1
        #print(f"Model {i} at epoch {checkpoint['epoch']} test results: accuracy: {test_accuracy*100}% f1: {test_f1}")
        accuracies.append(test_accuracy)
        f1s.append(test_f1)
        print(test_confusion)
        
    return round(np.mean(accuracies)*100, ), round(np.mean(f1s), 2)
    # print(f"Average accuracy: {round(np.mean(accuracies), 2)}%")
    # print(f"Average f1: {round(np.mean(f1s), 2)}")


In [15]:
params = (0.5, 50, 0.001, 32, 2)
accuracy, f1 = evaluate(params)
print(f"10-fold average accuracy: {accuracy}%, F1: {f1}")

0it [00:00, ?it/s]

1it [00:12, 12.33s/it]

[[25  1]
 [ 2  6]]


2it [00:22, 10.99s/it]

[[23  3]
 [ 2  6]]


...

[[24  2]
 [ 4  4]]


9it [02:03, 15.99s/it]

[[28  0]
 [ 1  5]]


10it [02:20, 14.09s/it]

[[24  3]
 [ 1  9]]
10-fold average accuracy: 91%, F1: 0.81


In [13]:
params_nn ={
    'dropout': [x/10 for x in list(range(0, 10, 3))],
    'hidden_size': list(range(0, 101, 25))[1:],
    'learning_rate': [0.01, 0.001, 0.0001, 1e-05],
    'batch_size': [2*2**x for x in range(2, 6)],
    'n_hidden': list(range(1, 5, 1))
}
parameter_expansion = list(product(*params_nn.values()))
print(len(parameter_expansion))

1024


In [14]:
results = {}
for i, p in tqdm(enumerate(parameter_expansion)):
    dropout, hidden_size, learning_rate, batch_size, n_hidden = p
    accuracy, f1, confusion = evaluate(p)
    model_performance = {"dropout": dropout, "hidden_size": hidden_size, "learning_rate": learning_rate, 
              "batch_size": batch_size, "n_hidden": n_hidden, "accuracy": accuracy, "f1": f1}
    results[i] = model_performance
    print("Confusion matrix:\n", confusion)
    print(model_performance)

0it [00:00, ?it/s]

[[25  1]
 [ 2  6]]


[[22  4]
 [ 1  7]]


2it [00:33, 16.53s/it]
0it [00:33, ?it/s]


KeyboardInterrupt: 

In [ ]:
results_dataframe = pd.DataFrame.from_dict(results)